In [ ]:
from __future__ import absolute_import, division
from IPython.display import clear_output
from BashColors import C

import glob, os, shutil
from os.path import *
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'

contentPath=os.getcwd()

import numpy as np
np.set_printoptions(precision=1)
import tensorflow as tf
assert tf.__version__.startswith('2')
import tflite_model_maker
from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader
import matplotlib.pyplot as plt

print(f'tf version: {C.BIBlue}{tf.__version__}{C.ColorOff}')
print(f'tflite version: {C.BIBlue}{tflite_model_maker.__version__}')

In [ ]:
import cv2
from time import sleep

def resizeImage(thisPath):
    print(f'{C.BIPurple}{thisPath}{C.ColorOff}')
    img = cv2.imread(thisPath)
    img = tf.image.resize(
        img, size=(224,224), method=tf.image.ResizeMethod.BILINEAR,
        preserve_aspect_ratio=False,
        antialias=False, name=None
    )
    img=np.copy(img)
    cv2.imwrite(thisPath, img)
    sleep(0.2)
    print(img.shape)
    
os.chdir(contentPath)
imageList=[]
initialList=[]
globList=[]

globList=glob.glob('**', recursive=True)
for pth in globList:
    fullPath=abspath(pth)
    # print(fullPath)
    initialList.append(fullPath)
    if fullPath.endswith('.png'):
        imageList.append(fullPath)
        
for item in imageList:
    # resizeImage(item)
    pass

In [ ]:
# Split it to training data (80%), validation data (10%, optional) and testing data (10%).

data = DataLoader.from_folder('DataGenerator')
print(f'data files: {C.BIPurple}{len(data)}{C.ColorOff}')
train_data, validation_data = data.split(0.8)
print(f'train_data: {C.BIPurple}{len(train_data)}{C.ColorOff}')
print(f'validation_data: {C.BIPurple}{len(validation_data)}{C.ColorOff}\n')

testData = DataLoader.from_folder('images')
test_data = testData
print(f'test_data: {C.BIPurple}{len(test_data)}{C.ColorOff}')

In [ ]:
plt.figure(figsize=(12, 12))
for i, (image, label) in enumerate(train_data.gen_dataset().unbatch().take(3)):
  plt.subplot(3, 3, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

In [ ]:
BATCH_SIZE =8
EPOCHS = 10
DROPOUT_RATE = 0.5
LEARNING_RATE = 0.001

model = image_classifier.create(
    train_data = train_data,
    validation_data = validation_data,
    model_spec = model_spec.get('mobilenet_v2'),
    learning_rate = LEARNING_RATE,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    dropout_rate = DROPOUT_RATE,
    use_augmentation = True, # default=False
    use_hub_library = True,  # default=True
    momentum = 0.9, # Only used when use_hub_library is True
    shuffle = True,
    train_whole_model = False
)

### Step 3: Evaluate the Customized Model

Evaluate the result of the model, get the loss and accuracy of the model.

In [ ]:
loss, accuracy = model.evaluate(test_data)

We could plot the predicted results in 100 test images. Predicted labels with red color are the wrong predicted results while others are correct.

### Step 4: Export to TensorFlow Lite Model

Convert the trained model to TensorFlow Lite model format with [metadata](https://www.tensorflow.org/lite/convert/metadata) so that you can later use in an on-device ML application. The label file and the vocab file are embedded in metadata. The default TFLite filename is `model.tflite`.

In many on-device ML application, the model size is an important factor. Therefore, it is recommended that you apply quantize the model to make it smaller and potentially run faster.
The default post-training quantization technique is full integer quantization for the image classification task.

See [example applications and guides of image classification](https://www.tensorflow.org/lite/models/image_classification/overview#example_applications_and_guides) for more details about how to integrate the TensorFlow Lite model into mobile apps.

This model can be integrated into an Android or an iOS app using the [ImageClassifier API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/image_classifier) of the [TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview).

In [ ]:
# model.export(export_dir='.')

exportList = [ExportFormat.TFLITE,
              ExportFormat.LABEL,
              ExportFormat.SAVED_MODEL]
float16Config = QuantizationConfig.for_float16()
# model.export(export_dir='.', export_format = ExportFormat.LABEL)
model.export(export_dir = contentPath,
             tflite_filename = 'Defcon4_fp16.tflite',
             export_format = exportList,
             config=float16Config)

In [ ]:
model.evaluate_tflite('Defcon4_fp16.tflite', test_data)

In [ ]:
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'blue'
  else:
    return 'black'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(224/4, 224/4))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
    ax = plt.subplot(11, 11, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image.numpy(), cmap=plt.cm.gray)
    predict_label = predicts[i][0][0]
    color = get_label_color(predict_label,
                            test_data.index_to_label[label.numpy()])
    ax.xaxis.label.set_fontsize(24)
    ax.xaxis.label.set_color(color)
    plt.xlabel('%s' % predict_label)
plt.show()

In [ ]:
# print(os.getcwd())
import glob
newList=[]
globList=[]
globList=glob.glob('**', recursive=True)

for pth in globList:
    fullPath=abspath(pth)
    if not fullPath in initialList:
        newList.append(fullPath)
        if isdir(fullPath):
            print(f'{C.BIBlue}{fullPath}{C.ColorOff}')
        elif isfile(fullPath):
            print(fullPath)

In [ ]:
# q

## Advanced Usage

The `create` function is the critical part of this library. It uses transfer learning with a pretrained model similar to the [tutorial](https://www.tensorflow.org/tutorials/images/transfer_learning).

The `create` function contains the following steps:

1.   Split the data into training, validation, testing data according to parameter `validation_ratio` and `test_ratio`. The default value of `validation_ratio` and `test_ratio` are `0.1` and `0.1`.
2.   Download a [Image Feature Vector](https://www.tensorflow.org/hub/common_signatures/images#image_feature_vector) as the base model from TensorFlow Hub. The default pre-trained model is  EfficientNet-Lite0.
3.   Add a classifier head with a Dropout Layer with `dropout_rate` between head layer and pre-trained model. The default `dropout_rate` is the default `dropout_rate` value from [make_image_classifier_lib](https://github.com/tensorflow/hub/blob/master/tensorflow_hub/tools/make_image_classifier/make_image_classifier_lib.py#L55) by TensorFlow Hub.
4.   Preprocess the raw input data. Currently, preprocessing steps including normalizing the value of each image pixel to model input scale and resizing it to model input size.   EfficientNet-Lite0 have the input scale `[0, 1]` and the input image size `[224, 224, 3]`.
5.   Feed the data into the classifier model. By default, the training parameters such as training epochs, batch size, learning rate, momentum are the default values from [make_image_classifier_lib](https://github.com/tensorflow/hub/blob/master/tensorflow_hub/tools/make_image_classifier/make_image_classifier_lib.py#L55) by TensorFlow Hub. Only the classifier head is trained.


In this section, we describe several advanced topics, including switching to a different image classification model, changing the training hyperparameters etc.


## Customize Post-training quantization on the TensorFLow Lite model


[Post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) is a conversion technique that can reduce model size and inference latency, while also improving CPU and hardware accelerator inference speed, with a little degradation in model accuracy. Thus, it's widely used to optimize the model.


Model Maker library applies a default post-training quantization techique when exporting the model. If you want to customize post-training quantization, Model Maker supports multiple post-training quantization options using [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig) as well. Let's take float16 quantization as an instance. First, define the quantization config.

In [ ]:
config = QuantizationConfig.for_float16()

model.export(export_dir =contentPath,
             tflite_filename = 'Defcon4_fp16.tflite',
             quantization_config=config)

In [ ]:
loss, accuracy = model.evaluate(test_data)

### Change your own custom model

If we'd like to use the custom model that's not in TensorFlow Hub, we should create and export [ModelSpec](https://www.tensorflow.org/hub/api_docs/python/hub/ModuleSpec) in TensorFlow Hub.

Then start to define `ModelSpec` object like the process above.

# Read more

You can read our [image classification](https://www.tensorflow.org/lite/examples/image_classification/overview) example to learn technical details. For more information, please refer to:

*   TensorFlow Lite Model Maker [guide](https://www.tensorflow.org/lite/guide/model_maker) and [API reference](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker).
*   Task Library: [ImageClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/image_classifier) for deployment.
*   The end-to-end reference apps: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/android), [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/ios), and [Raspberry PI](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/raspberry_pi).

